# Analysis of Chicago Police Homicide Clearance Data

In [5]:
import re
import pandas as pd
from src import schemas


## Data import and cleaning

In [10]:
rename_columns = lambda df: df.rename(
    columns=lambda col: re.sub(
        r"\s+", "_", re.sub(r"[^A-z]+", " ", col.strip()).lower().strip()
    )
)

status = (
    pd.read_excel(
        "input/18731-P744333_Homicide_status_2010-YTD.xlsx",
        sheet_name=1,
        parse_dates=["Injury Date", "Death Date", "CompStat Date", "Date Cleared"],
    )
    .pipe(rename_columns)
    .replace(
        {
            "cleared_i": {"Y": True, "N": False},
            "victim_sex": {"M": "male", "F": "female", "X": "unknown/other"},
        }
    )
    .rename(columns={"homicide": "homicide_no"})
    .query("injury_date >= '2010-01-01'")
    .pipe(schemas.status_schema)
    .set_index("rd")
)

victims = (
    pd.read_csv(
        "input/Violence_Reduction_-_Victims_of_Homicides_and_Non-Fatal_Shootings.csv",
        parse_dates=["DATE"],
    )
    .pipe(rename_columns)
    .query("victimization_primary == 'HOMICIDE'")
    .rename(columns={"race": "victim_race"})[["case_number", "victim_race"]]
    .replace(
        {
            "victim_race": {
                "BLK": "black",
                "WHI": "white",
                "WWH": "hispanic",
                "WBH": "hispanic",
                "API": "api",
                "I": "indian",
                "UNKNOWN": "unknown",
            }
        }
    )
    .pipe(schemas.victims_schema)
    .set_index("case_number")
)

df = status.join(victims)
df


,homicide_no,injury_date,death_date,compstat_date,address_block_level,homicide_location_descr,victim_sex,victim_age,cleared_i,cleared_exceptionally_by,date_cleared,victim_race
HS101859,10010002,2010-01-02 21:44:00,2010-01-02 21:44:00,2010-01-02 21:44:00,57XX S LOOMIS BLVD,AUTO,male,27.0,False,NaN,NaT,hispanic
HS102143,10010001,2010-01-02 16:17:00,2010-01-02 16:17:00,2010-01-02 16:17:00,110XX S LOWE AVE,HOUSE,female,61.0,True,NaN,2010-01-05,black
HS102573,10010003,2010-01-02 20:30:00,2010-01-03 15:04:00,2010-01-03 15:04:00,59XX S WOLCOTT AVE,HOUSE,male,49.0,False,NaN,NaT,black
HS103194,10010004,2010-01-03 14:12:00,2010-01-04 00:37:00,2010-01-04 00:37:00,77XX S CHAMPLAIN AVE,STREET,male,21.0,True,NaN,2010-02-03,black
HS104874,10010005,2010-01-04 17:15:00,2010-01-04 17:15:00,2010-01-04 17:15:00,29XX W 61ST ST,STREET,male,17.0,True,NaN,2010-01-15,hispanic
...,...,...,...,...,...,...,...,...,...,...,...,...
JF219389,22040180,2022-04-25 11:47:00,2022-04-25 12:16:00,2022-04-25 11:47:00,37XX W CHICAGO AVE,GAS STATION,male,32.0,False,NaN,NaT,black
JF221655,22040182,2022-04-27 09:29:00,2022-04-27 10:20:00,2022-04-27 09:29:00,65XX W DIVERSEY AVE,PARKING LOT,female,47.0,True,NaN,2022-04-28,white
JF223239,22040183,2022-04-28 15:43:00,2022-04-28 16:08:00,2022-04-28 15:43:00,16XX E 67TH ST,VESTIBULE,male,30.0,False,NaN,NaT,black
JF224596,22040184,2022-04-29 17:50:00,2022-04-29 18:24:00,2022-04-29 17:50:00,3XX S KILPATRICK AVE,HALLWAY,male,68.0,False,NaN,NaT,black


,homicide_no,injury_date,death_date,compstat_date,address_block_level,homicide_location_descr,victim_sex,victim_age,cleared_i,cleared_exceptionally_by,date_cleared,victim_race
HS101859,10010002,2010-01-02 21:44:00,2010-01-02 21:44:00,2010-01-02 21:44:00,57XX S LOOMIS BLVD,AUTO,male,27.0,False,NaN,NaT,hispanic
HS102143,10010001,2010-01-02 16:17:00,2010-01-02 16:17:00,2010-01-02 16:17:00,110XX S LOWE AVE,HOUSE,female,61.0,True,NaN,2010-01-05,black
HS102573,10010003,2010-01-02 20:30:00,2010-01-03 15:04:00,2010-01-03 15:04:00,59XX S WOLCOTT AVE,HOUSE,male,49.0,False,NaN,NaT,black
HS103194,10010004,2010-01-03 14:12:00,2010-01-04 00:37:00,2010-01-04 00:37:00,77XX S CHAMPLAIN AVE,STREET,male,21.0,True,NaN,2010-02-03,black
HS104874,10010005,2010-01-04 17:15:00,2010-01-04 17:15:00,2010-01-04 17:15:00,29XX W 61ST ST,STREET,male,17.0,True,NaN,2010-01-15,hispanic
...,...,...,...,...,...,...,...,...,...,...,...,...
JF219389,22040180,2022-04-25 11:47:00,2022-04-25 12:16:00,2022-04-25 11:47:00,37XX W CHICAGO AVE,GAS STATION,male,32.0,False,NaN,NaT,black
JF221655,22040182,2022-04-27 09:29:00,2022-04-27 10:20:00,2022-04-27 09:29:00,65XX W DIVERSEY AVE,PARKING LOT,female,47.0,True,NaN,2022-04-28,white
JF223239,22040183,2022-04-28 15:43:00,2022-04-28 16:08:00,2022-04-28 15:43:00,16XX E 67TH ST,VESTIBULE,male,30.0,False,NaN,NaT,black
JF224596,22040184,2022-04-29 17:50:00,2022-04-29 18:24:00,2022-04-29 17:50:00,3XX S KILPATRICK AVE,HALLWAY,male,68.0,False,NaN,NaT,black
